# Tensor

Tensor 是深度学习框架中极为基础的概念，也是 PyTroch、TensorFlow 中最重要的知识点之一，它是一种数据的存储和处理结构。

几种数据类型：

- 标量，也称 Scalar，是一个只有大小，没有方向的量，比如 1.8、e、10 等。
- 向量，也称 Vector，是一个有大小也有方向的量，比如 (1,2,3,4) 等。
- 矩阵，也称 Matrix，是多个向量合并在一起得到的量，比如 [(1,2,3),(4,5,6)] 等。
- 张量，也称 Tensor

从标量、向量和矩阵的关系来看，它们就是不同“**维度**”的 Tensor ？

更准确地使用 Rank（秩）来表示这种“维度”，比如标量，就是 Rank 为 0 阶的 Tensor；向量就是 Rank 为 1 阶的 Tensor；矩阵就是 Rank 为 2 阶的 Tensor；还有 Rank 大于 2 的 Tensor。

In [1]:
import torch

import numpy as np

## 数据类型

一些常用类型

<img src="https://static001.geekbang.org/resource/image/e6/08/e6af6a3b2172ee08db8c564146ae2108.jpg?wh=1680x933" width="50%" />

## 创建

直接创建

```torch.tensor(data, dtype=None, device=None,requires_grad=False)```

- data，是要传入模型的数据。PyTorch 支持通过 list、 tuple、numpy array、scalar 等多种类型进行数据传入，并转换为 tensor；
- dtype，它声明了需要返回的 Tensor 的类型；
- device，这个参数指定了数据要返回到的设备；
- requires_grad，用于说明当前量是否需要在计算中保留对应的梯度信息。

  > requires_grad 的设置要灵活处理：
  >- 如果是训练过程就要设置为 true，目的是方便求导、更新参数。而到了验证或者测试过程；
  >- 如果目的是检查当前模型的泛化能力，就要设置成 Fasle，避免这个参数根据 loss 自动更新。

从 Numpy 创建

```
torch.from_numpy(ndarry)
```

创建特殊形式的 Tensor

```
torch.zeros(*size, dtype=None...)
torch.ones(size, dtype=None...)
```

创建单位矩阵 Tensor：（单位矩阵是指主对角线上的元素都为 1 的矩阵）

```
torch.eye(size, dtype=None...)
```

创建随机矩阵 Tensor ，在 PyTorch 中有几种较为经常使用的随机矩阵创建方式，分别如下：

```
torch.rand(size)
torch.randn(size)
torch.normal(mean, std, size)
torch.randint(low, high, size）
```

- torch.rand 用于生成数据类型为浮点型且维度指定的随机 Tensor，随机生成的浮点数据在 0~1 区间均匀分布。
- torch.randn 用于生成数据类型为浮点型且维度指定的随机 Tensor，随机生成的浮点数的取值满足均值为 0、方差为 1 的标准正态分布。
- torch.normal 用于生成数据类型为浮点型且维度指定的随机 Tensor，可以指定均值和标准差。
- torch.randint 用于生成随机整数的 Tensor，其内部填充的是在[low,high) 均匀生成的随机整数。


## Tensor 的转换

Int 与 Tensor 的转换：

通过 torch.tensor 将一个数字（或者标量）转换为 Tensor，又通过 item() 函数，将 Tensor 转换为数字（标量），item() 函数的作用就是将 Tensor 转换为一个 python number。

In [2]:
a = torch.tensor(1)
b = a.item()
print(a)
print(b)

tensor(1)
1


list 与 Tensor 的转换：

使用 torch.tensor 就可以将其转换为 Tensor 了。而还原回来的过程要多一步，需要先将 Tensor 转为 NumPy 结构，之后再使用 tolist() 函数得到 list。

In [3]:
a = [1, 2, 3]
b = torch.tensor(a)
c = b.numpy().tolist()

print(b)
print(c)

tensor([1, 2, 3])
[1, 2, 3]


NumPy 与 Tensor 的转换：使用 torch.tensor

In [4]:
np_data = np.arange(6).reshape((2, 3))

torch_data = torch.tensor(np_data)
# torch_data = torch.from_numpy(np_data)

tensor2array = torch_data.numpy()

print(
    "\nnumpy array:\n", np_data,
    "\ntorch tensor:\n", torch_data,
    "\ntensor to array:\n", tensor2array,
)


numpy array:
 [[0 1 2]
 [3 4 5]] 
torch tensor:
 tensor([[0, 1, 2],
        [3, 4, 5]]) 
tensor to array:
 [[0 1 2]
 [3 4 5]]


In [5]:
# torch.tensor(np_data)
torch_data

tensor([[0, 1, 2],
        [3, 4, 5]])

CPU 与 GPU 的 Tensor 之间的转换：

```
CPU->GPU: data.cuda()
GPU->CPU: data.cpu()
```

## Tensor 的常用操作

获取形状

使用 shape 或 size 来获取。两者的不同之处在于，shape 是 Tensor对象 的一个属性，而 size() 则是一个 Tensor对象 拥有的方法

In [6]:
a = torch.zeros(2, 3, 5)
print(a.shape)
print(a.size())

torch.Size([2, 3, 5])
torch.Size([2, 3, 5])


知道了 Tensor 的形状，我们就能知道这个 Tensor 所包含的元素的数量了。具体的计算方法就是直接将所有维度的大小相乘，比如上面的 Tensor a 所含有的元素的个数为 2 * 3 * 5=30 个。

In [7]:
a.numel()

30

矩阵转秩 (维度转换）

在 PyTorch 中有两个函数： permute() 和 transpose() 可以用来实现矩阵的转秩，或者说交换不同维度的数据。
比如在调整卷积层的尺寸、修改 channel 的顺序、变换全连接层的大小的时候，就要用到它们。

函数 permute ，可以对任意高维矩阵进行转置，但只有 tensor.permute() 这个调用方式。

函数 transpose，不同于 permute，它每次只能转换两个维度，或者说交换两个维度的数据。

<img src="https://static001.geekbang.org/resource/image/02/84/025985c8635f3896d45d15e1ea381c84.jpg" width="30%" />

In [8]:
x = torch.rand(2,3,5)
print(x.shape)

# 2 表示原来第二个维度现在放在了第零个维度；
# 同理 1 表示原来第一个维度仍旧在第一个维度；
# 0 表示原来第 0 个维度放在了现在的第 2 个维度
# 最后，形状就变成 [5,3,2]
x = x.permute(2,1,0)
print(x.shape)


torch.Size([2, 3, 5])
torch.Size([5, 3, 2])


In [9]:
x = torch.rand(2,3,4)
print(x.shape)
x = x.transpose(1,0)
print(x.shape)

torch.Size([2, 3, 4])
torch.Size([3, 2, 4])


In [10]:
x = torch.rand(2,3,5)
print(x.shape)

x = x.transpose(0, 2)
print(x.shape)

torch.Size([2, 3, 5])
torch.Size([5, 3, 2])


经过了 transpose 或者 permute 处理之后的数据，变得不再连续了。内存虽然没有变化，但是的数据第 0 和第 2 维的数据发生了交换，现在的第 0 维是原来的第 2 维，所以 Tensor 都会变得不再连续。

不连续就不连续呗，好像也没啥影响吧？是这样的吗？接着看。

形状变换

在 PyTorch 中有两种常用的改变形状的函数，分别是 view 和 reshape。


In [11]:
x = torch.randn(4, 4)
print(x.shape)
x = x.view(2,8)
print(x.shape)
x

torch.Size([4, 4])
torch.Size([2, 8])


tensor([[-0.7802,  1.0232, -0.4342, -0.4089,  0.9112, -0.5994, -1.6719, -0.2978],
        [-0.7073, -0.2912,  0.2460, -0.1393,  0.7928,  0.3846,  1.9325, -1.9571]])

In [12]:
x = x.permute(1,0)
print(x.shape)

# x.view(4, 4)

# RuntimeError: view size is not compatible with input tensor's size and stride 
# (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

torch.Size([8, 2])


抛出错误是由于经过 permute 后，将第 0 和第 1 维度的数据进行了变换，跨连续子空间了，而 view 无法处理内存不连续的 Tensor 。

而 reshape 可以很好地工作，因为 reshape 相当于进行了两步操作，先把 Tensor 在内存中捋顺了，然后再进行 view 操作。

In [13]:
x = x.reshape(4, 4)
x

tensor([[-0.7802, -0.7073,  1.0232, -0.2912],
        [-0.4342,  0.2460, -0.4089, -0.1393],
        [ 0.9112,  0.7928, -0.5994,  0.3846],
        [-1.6719,  1.9325, -0.2978, -1.9571]])

增减维度

有时候我们需要对 Tensor 增加或者删除某些维度，比如删除或者增加图片的几个通道。PyTorch 提供了 squeeze() 和 unsqueeze() 函数解决这个问题。

- squeeze()：如果 dim 指定的维度的值为 1，则将该维度删除，若指定的维度值不为 1，则返回原来的 Tensor。
- unsqueeze()：这个函数主要是对数据维度进行扩充。给指定位置加上维数为 1 的维度。

In [14]:
x = torch.rand(2,1,3)
print(x.shape)

y = x.squeeze(1)
print(y.shape)

# 这里失败了：因为 y 此刻的第 1 维度的大小为 3，suqeeze 不能删除。
z = y.squeeze(1)
print(z.shape)

torch.Size([2, 1, 3])
torch.Size([2, 3])
torch.Size([2, 3])


In [15]:
x = torch.rand(2,1,3)
y = x.unsqueeze(2)
y.shape

torch.Size([2, 1, 1, 3])

## 小结

把 NumPy 和 Tensor 做对比，它们之间有很多共通的内容，两者都是数据的表示形式，都可以看作是科学计算的通用工具。但是 NumPy 和 Tensor 的用途是不一样的，NumPy 不能用于 GPU 加速，Tensor 则可以。
